In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import nltk

In [2]:
df = pd.read_csv('../data/memes.csv', sep=';')

In [3]:
df['bag_of_tagram'] = df.content.apply(lambda text: ' '.join(map(lambda par: par[1], nltk.pos_tag(word_tokenize(text)))))

In [4]:
df.sample(3)

,meme,content,bag_of_tagram
13,Ancient-Aliens,Something,VBG
30,First-World-Problems,"I spend all my money on food, still hungry","PRP VBP DT PRP$ NN IN NN , RB JJ"
8,All-The-Things,Wax all the hair,NNP PDT DT NN


In [5]:
meme_enc = LabelEncoder().fit(df.meme.unique())

In [6]:
y = meme_enc.transform(df.meme)

In [7]:
tfidf = TfidfVectorizer(ngram_range=(1, 2)).fit(df.bag_of_tagram)

In [8]:
X = tfidf.transform(df.bag_of_tagram)

In [9]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [10]:
def qual_meme(content):
    return meme_enc.inverse_transform(knn.predict(tfidf.transform([content])))[0]

In [11]:
qual_meme('what if Rogerio is the best CS player?')

'Bad-Luck-Brian'

In [12]:
qual_meme('Plays CS, gets killed')

'Bad-Luck-Brian'

In [13]:
qual_meme('To be or not to be?')

'Conspiracy-Keanu'